In [ ]:
"""
Python Intro to Homomorphic Encryption
--
This Jupyter notebook covers the basics of choosing a cryptosystem, features of homomorphic encryption,
and how to get started.

The code is in Python 3.
"""

## What is it

"""
Homomorphic encryption allows you to encrypt a number in a way that other services can run arithmetic and logical
operations on them.

For example, if you wanted to exchange currencies at the bank, but first wanted to know the exchange rate and fee
for the transaction, typically you send the amount to the banks, or they send you a formula to predict their
charges. With homomorphic encryption, you could send an encrypted dollar-amount to the bank, they would run their
exchange rate / fee algorithm, and then you could decrypt the result, without revealing how much money you were
considering transfering / exchanging.

I've heard some ideas about applying this to medical scans... maybe a service could diagnose you without having to see
your un-encrypted private data.

My goal was to use homomorphic encryption to protect a client's geolocation. You may have other applications in mind.
"""

## Partial, Somewhat, Fully

"""
Homomorphic encryption systems have been researched for years. They fall into three categories:

- partially homomorphic: only covers some arithmetic / logic operations (for example, addition not multiplication)
- somewhat homomorphic: covers only a few sequential operations before cyphertext is too large / noisy to decypher
- fully homomorphic encryption: first achieved in 2009, allows all logic operations
"""

## Choosing a homomorphic encryption system

"""
When you search GitHub for "homomorphic encryption", you will find libraries in several programming languages.
You can see several different cryptosystems, with different levels (partial, somewhat, fully homomorphic) and
different levels of covering those features.

There are fully homomorphic libraries in C++, but I want libraries with JavaScript implementations that work
in the browser. So I picked the Paillier partially-homomorphic cryptosystem.
"""


In [2]:
# pip install phe

"""
When you run this on the command line, you will have installed a module for the Paillier cryptosystem,
called "phe" for partially homomorphic encryption.

If you don't have pip installed yet, read: https://pip.pypa.io/en/stable/installing/

The following line checks that we can import the module:
"""

from phe import paillier

In [35]:
# Let's start with the exchange rate example.

# generate the public and private keys
# if this part is confusing, read https://blog.vrypan.net/2013/08/28/public-key-cryptography-for-non-geeks/
keypair = paillier.generate_paillier_keypair()
publicKey = keypair[0]
privateKey = keypair[1]

print(publicKey)
print(privateKey)

<PaillierPublicKey c4067d8a12>
<PaillierPrivateKey for <PaillierPublicKey c4067d8a12>>


In [36]:
# let's protect our secret dollar amount
# note that Paillier only supports positive integers

dollarAmount = 10500
secretDollarAmount = publicKey.encrypt(dollarAmount)

print(secretDollarAmount)
print(secretDollarAmount.ciphertext())

1849385480737683652751246054269062436484215425592527569812047577066503857715088989890230236613770315674222028195275198347944912085811060967845706272471783985108496242787626277347480611684661726859014297872238001188814286421543493776178974952381024123098555883402313223873615382523482469923729258402265875507457516187608553542322484713362179340274058803895907290706669333049052785409581975207720302003420468543113568840155344105994360410635837195819693306919977650057952336777084152907400027259563034007457296029164238292487453405302579197542156878523742341315605093343449568350363974205272336857347164172447078534884579763538591997339104270539887995485296293379861570879819438239776886473422158974604385149433112342339638800853557955565176333441432717820125499991067416867185023839959944098956466867965647232522355924895567119488842853570206505028448797083633865787643238349599157951184674014973002796753753294293476117370316455304699153361823617890078320688006184822446468408633803058856931444621679

In [45]:
# that's a big number!
# but more importantly, the SketchyExchangeBank will not know how much money is involved
# the bank will need our PUBLIC key to move its numbers into our crypto-space

def SketchyExchangeBank(customerKey, customerSecretDollars):
    # secret formula: $19 base fee, then 2.42 / dollar
    baseFee = 19
    exchangeRate = 2.42 # we are able to multiply by a decimal
    
    encryptedFee = customerKey.encrypt(baseFee)
    # the libary facilitates add / subtract / multiply operations within Paillier cryptosystem
    estimate = customerSecretDollars - encryptedFee
    estimate = estimate * exchangeRate
    return estimate

# what comes back is another EncryptedNumber
response = SketchyExchangeBank(publicKey, secretDollarAmount)
print(response)
print(response.ciphertext())

3916493590037648894635146493468866363700969603594037225854746983622640729972484208421354189193249303823305257619018532524309649994437812026848562547718180895046958643552476632578217724412261222627841938190531123447806105395323987091690044887365837413740598758271419032077023917575073679196936482485670984230454199999475360786471122231725812262904269059572548022859283325055955632559394049053681399451944122689537892142202804865653315260436410818118346251164932963408326758162938051074928299476633777767774736716878827273167997873948489732889911564158971142372769477418296847592132901198072815759005087209794981604339307109736428450491712601302322454914317722399097264934953524755122769089167998691940884108518572977331650648754121991987148632915227852007833711064066260051030998841774020573977318218684599444299862192462956043506945625049470892117925275280608530747247900905850412497893632893800995615950361272535267532498837522710162244256955839768603178454638413486484095284093886372641766898045535

In [46]:
# Back on the client side, let's decrypt the result from the bank
print(privateKey.decrypt(response))

25364.02
